<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
これは <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>（<a href="https://sebastianraschka.com">Sebastian Raschka</a>著）の補足コードである<br>
<br>コードリポジトリ: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp?1" width="100px"></a>
</td>
</tr>
</table>

# 第7章: 命令に従うファインチューニング

In [1]:
from importlib.metadata import version

pkgs = [
    "numpy",       # PyTorch & TensorFlow dependency
    "matplotlib",  # Plotting library
    "tiktoken",    # Tokenizer
    "torch",       # Deep learning library
    "tqdm",        # Progress bar
    "tensorflow",  # For OpenAI's pretrained weights
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

numpy version: 2.0.2
matplotlib version: 3.10.0
tiktoken version: 0.8.0
torch version: 2.5.1+cu124
tqdm version: 4.67.1
tensorflow version: 2.18.0


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/overview.webp?1" width=500px>

## 7.1 インストラクションファインチューニングの導入

- 第5章で、LLMの事前学習では単語を1つずつ生成する手順を通じて学習することを見た
- したがって、事前学習済みのLLMはテキストの補完に優れているが、命令に従うのは得意ではない
- この章では、LLMが命令によりうまく従えるように学習する

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/instruction-following.webp" width=500px>

- この章の内容は下図のようにまとめられる

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/chapter-overview-1.webp?1" width=500px>

## 7.2 スーパーバイザードなインストラクションファインチューニング用データセットの準備

- この章で使用するインストラクション用データセットを用意してある

In [2]:
import json
import os
import urllib


def download_and_load_file(file_path, url):

    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)

    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    return data

file_path = "instruction-data.json"
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))

Number of entries: 1100


- 上で読み込んだ `data` リストの各要素は、次の形式をとる辞書である

In [3]:
print("Example entry:\n", data[50])

Example entry:
 {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}


- `'input'` フィールドは空である場合がある:

In [4]:
print("Another example entry:\n", data[999])

Another example entry:
 {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}


- インストラクションファインチューニングは、しばしば「スーパーバイザードインストラクションファインチューニング」と呼ばれる。これは、入力と出力のペアが明示的に与えられたデータセットを用いてモデルを訓練するためである
- エントリをLLMへの入力として整形する方法はいくつかある。下図は、Alpaca (https://crfm.stanford.edu/2023/03/13/alpaca.html) と Phi-3 (https://arxiv.org/abs/2404.14219) というLLMの訓練に用いられた2つの例示的なフォーマットである

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/prompt-style.webp?1" width=500px>

- この章では、Alpacaスタイルのプロンプトフォーマットを使用する。これは元々インストラクションファインチューニングに用いられていたプロンプトテンプレートである
- 以下では、LLMに入力として渡すためのテキストを整形している

In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

- `input` フィールドがある場合の整形例は以下である

In [6]:
model_input = format_input(data[50])
desired_response = f"\n\n### Response:\n{data[50]['output']}"

print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Identify the correct spelling of the following word.

### Input:
Ocassion

### Response:
The correct spelling is 'Occasion.'


- `input` フィールドがない場合の整形例は以下である

In [7]:
model_input = format_input(data[999])
desired_response = f"\n\n### Response:\n{data[999]['output']}"

print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is an antonym of 'complicated'?

### Response:
An antonym of 'complicated' is 'simple'.


- 最後に、次のセクションでPyTorchのデータローダーを用意する前に、データセットを訓練・バリデーション・テストの3つに分割する

In [8]:
train_portion = int(len(data) * 0.85)  # 85% for training
test_portion = int(len(data) * 0.1)    # 10% for testing
val_portion = len(data) - train_portion - test_portion  # Remaining 5% for validation

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

In [9]:
print("Training set length:", len(train_data))
print("Validation set length:", len(val_data))
print("Test set length:", len(test_data))

Training set length: 935
Validation set length: 55
Test set length: 110


## 7.3 訓練バッチへのデータ整理

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/chapter-overview-2.webp?1" width=500px>

- このデータセットのバッチングは、下図にまとめたように複数のステップで行う

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/detailed-batching.webp?1" width=500px>

- まず、第6章の `SpamDataset` と同様に、データセット内の入力をあらかじめトークナイズする `InstructionDataset` クラスを実装する

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/pretokenizing.webp" width=500px>

In [10]:
import torch
from torch.utils.data import Dataset


class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data

        # Pre-tokenize texts
        self.encoded_texts = []
        for entry in data:
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

    def __getitem__(self, index):
        return self.encoded_texts[index]

    def __len__(self):
        return len(self.data)

- 第6章と同様に、訓練を加速するために複数のサンプルをバッチ化する必要がある。このとき、入力の長さを揃えるためにパディングが必要になる
- 前章と同様に、パディングトークンとして `<|endoftext|>` （ID 50256）を使う

In [11]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


- 第6章では、データセット内のすべてのサンプルを同じ長さにパディングした
  - 今回は、バッチごとに異なる最大長に合わせてパディングを行うためのカスタム`collate`関数を作成する
  - これにより、各バッチ内のみ同じ長さにそろえる（バッチ間では長さが違う可能性がある）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/padding.webp" width=500px>

In [12]:
def custom_collate_draft_1(
    batch,
    pad_token_id=50256,
    device="cpu"
):
    # Find the longest sequence in the batch
    # and increase the max length by +1, which will add one extra
    # padding token below
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs
    inputs_lst = []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to batch_max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        # Via padded[:-1], we remove the extra padded token
        # that has been added via the +1 setting in batch_max_length
        # (the extra padding token will be relevant in later codes)
        inputs = torch.tensor(padded[:-1])
        inputs_lst.append(inputs)

    # Convert list of inputs to tensor and transfer to target device
    inputs_tensor = torch.stack(inputs_lst).to(device)
    return inputs_tensor

In [13]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]

batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print(custom_collate_draft_1(batch))

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/batching-step-4.webp?1" width=500px>

- 上の例ではLLMへの入力のみ返している。しかしLLMの学習にはターゲット値も必要である
- 事前学習と同様、ターゲットは1つ右にシフトされた入力であり、次のトークンを予測するように学習する

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/inputs-targets.webp?1" width=400px>

In [14]:
def custom_collate_draft_2(
    batch,
    pad_token_id=50256,
    device="cpu"
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets
        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs to tensor and transfer to target device
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)
    return inputs_tensor, targets_tensor

In [15]:
inputs, targets = custom_collate_draft_2(batch)
print(inputs)
print(targets)

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256, 50256, 50256, 50256],
        [    8,     9, 50256, 50256, 50256]])


- 次に `ignore_index` を導入して、パディングトークンIDを別の値に置き換える。この `ignore_index` の目的は、損失関数でパディング値を無視できるようにするためである

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/batching-step-5.webp?1" width=500px>

- 具体的には、トークンID 50256 を -100 に置き換える（以下の図を参照）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/ignore-index.webp?1" width=500px>

- （さらに、GPT-2がサポートするコンテキストサイズ1024トークンを超えるような長いデータを使う場合に備え、`allowed_max_length` も導入している）

In [16]:
def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
    allowed_max_length=None,
    device="cpu"
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets

        # New: Replace all but the first padding tokens in targets by ignore_index
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index

        # New: Optionally truncate to maximum sequence length
        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]
            targets = targets[:allowed_max_length]

        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs and targets to tensors and transfer to target device
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor

In [17]:
inputs, targets = custom_collate_fn(batch)
print(inputs)
print(targets)

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256,  -100,  -100,  -100],
        [    8,     9, 50256,  -100,  -100]])


- -100 に置き換えることでどうなるのかを見てみる
- ここでは分かりやすいように、2クラス分類（0と1）のタスクを想定する（第6章で扱ったような単純な例）
- 以下のようなロジット（モデルの最終出力）を考えた場合、損失は次のように計算される

In [18]:
logits_1 = torch.tensor(
    [[-1.0, 1.0],  # 1st training example
     [-0.5, 1.5]]  # 2nd training example
)
targets_1 = torch.tensor([0, 1])

loss_1 = torch.nn.functional.cross_entropy(logits_1, targets_1)
print(loss_1)

tensor(1.1269)


- もう1つサンプルを追加すると、損失値は当然変化する

In [19]:
logits_2 = torch.tensor(
    [[-1.0, 1.0],
     [-0.5, 1.5],
     [-0.5, 1.5]]  # New 3rd training example
)
targets_2 = torch.tensor([0, 1, 1])

loss_2 = torch.nn.functional.cross_entropy(logits_2, targets_2)
print(loss_2)

tensor(0.7936)


- このうち1つのサンプルのクラスラベルを-100に置き換えたらどうなるかを見てみる

In [20]:
targets_3 = torch.tensor([0, 1, -100])

loss_3 = torch.nn.functional.cross_entropy(logits_2, targets_3)
print(loss_3)
print("loss_1 == loss_3:", loss_1 == loss_3)

tensor(1.1269)
loss_1 == loss_3: tensor(True)


- 3つの訓練サンプルを使ったときの損失が、最初の2つのサンプルだけで計算した損失と同じになっている。つまり-100となっている例は無視されていることがわかる
- PyTorchの `cross_entropy(..., ignore_index=-100)` は、ラベルが-100の例を無視するようにデフォルトで設定されている
- -100の `ignore_index` を使うことで、追加で挿入したend-of-text（パディング）トークンを無視できる。一方、最初のend-of-text（パディング）トークンはLLMにとって応答の終了を示すために役立つので無視しない


- 実際には、命令部分に対応するターゲットトークンIDを無視することも一般的である（詳細は本章の演習として推奨）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/mask-instructions.webp?1" width=600px>

## 7.4 インストラクション用データセットに対するデータローダーの作成

- このセクションでは、`InstructionDataset` クラスと `custom_collate_fn` 関数を使って、訓練・バリデーション・テストデータのローダーを生成する

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/chapter-overview-3.webp?1" width=500px>

- 先ほどの `custom_collate_fn` では、バッチ単位でデバイス（GPUなど）にデータを移動している。これにより、トレーニングループで都度データを移動するよりも効率がよくなる（バックグラウンド処理として行われるため）。
- Pythonの標準ライブラリ `functools` の `partial` を使うことで、`device` 引数をあらかじめ指定した関数を生成できる

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note:
# Uncommenting the following lines will allow the code to run on Apple Silicon chips, if applicable,
# which is much faster than on an Apple CPU (as measured on an M3 MacBook Air).
# However, the resulting loss values may be slightly different.

#if torch.cuda.is_available():
#    device = torch.device("cuda")
#elif torch.backends.mps.is_available():
#    device = torch.device("mps")
#else:
#    device = torch.device("cpu")

print("Device:", device)

Device: cuda


In [22]:
from functools import partial

customized_collate_fn = partial(
    custom_collate_fn,
    device=device,
    allowed_max_length=1024
)

- 次に、これまでの章と同様の手順でデータローダーを生成する。ただし今回はバッチングの際に先ほどの `custom_collate_fn` を指定している

In [23]:
from torch.utils.data import DataLoader


num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_dataset = InstructionDataset(train_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

In [24]:
val_dataset = InstructionDataset(val_data, tokenizer)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

test_dataset = InstructionDataset(test_data, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

- 生成される入力バッチとターゲットバッチの次元を確認する

In [25]:
print("Train loader:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

Train loader:
torch.Size([8, 61]) torch.Size([8, 61])
torch.Size([8, 76]) torch.Size([8, 76])
torch.Size([8, 73]) torch.Size([8, 73])
torch.Size([8, 68]) torch.Size([8, 68])
torch.Size([8, 65]) torch.Size([8, 65])
torch.Size([8, 72]) torch.Size([8, 72])
torch.Size([8, 80]) torch.Size([8, 80])
torch.Size([8, 67]) torch.Size([8, 67])
torch.Size([8, 62]) torch.Size([8, 62])
torch.Size([8, 75]) torch.Size([8, 75])
torch.Size([8, 62]) torch.Size([8, 62])
torch.Size([8, 68]) torch.Size([8, 68])
torch.Size([8, 67]) torch.Size([8, 67])
torch.Size([8, 77]) torch.Size([8, 77])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 79]) torch.Size([8, 79])
torch.Size([8, 71]) torch.Size([8, 71])
torch.Size([8, 66]) torch.Size([8, 66])
torch.Size([8, 83]) torch.Size([8, 83])
torch.Size([8, 68]) torch.Size([8, 68])
torch.Size([8, 80]) torch.Size([8, 80])
torch.Size([8, 71]) torch.Size([8, 71])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 65]) torch.Size([8, 65])
torch.Size([8, 68]) torch.

- 上の出力からわかるように、すべてのバッチでバッチサイズは8だが、各バッチごとに長さが異なる
- バッチ内の先頭要素（1つ目のサンプル）の `<|endoftext|>` パディングトークン（ID: 50256）が含まれているか確認する

In [26]:
for inputs, targets in train_loader:
    print(inputs[0])
    break

tensor([... 50256, 50256, 50256])


- 同様に、ターゲットに -100 のプレースホルダトークンが含まれているかを目視確認する

In [27]:
print(targets[0])

tensor([  318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,   257,
         2882,   326, 20431, 32543,   262,  2581,    13,   198,   198, 21017,
        46486,    25,   198, 30003,  6525,   262,  6827,  1262,   257,   985,
          576,    13,   198,   198, 21017, 23412,    25,   198,   464,  5156,
          318,   845, 13779,    13,   198,   198, 21017, 18261,    25,   198,
          464,  5156,   318,   355, 13779,   355,   257,  4936,    13,   -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100], device='cuda:0')


## 7.7 応答の抽出と保存

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/chapter-overview-6.webp?1" width=500px>

- このセクションでは、次のセクションでスコアリングするためにテストセットの応答を保存する

In [ ]:
import psutil

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [ ]:
# このセルはオプションである。ノートブックを再起動し、セクション7.7だけを実行して前のコードを再実行しないようにするためのもの
import json
from tqdm import tqdm

file_path = "instruction-data-with-response.json"

with open(file_path, "r") as file:
    test_data = json.load(file)

def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

## 7.8 モデル応答へのスコアリング

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/chapter-overview-7.webp?1" width=500px>

- このセクションでは、外部ツール（例: Ollama）と Llama 3 を用いて、モデル応答をスコアリングするプロセスを示す

- まず、Ollamaが実行されているか確認する

In [ ]:
import subprocess

def query_model(prompt, model="llama3"):
    """Query the Llama 3 model (or another model) via Ollama."""
    cmd = [
        "ollama", "run", "--model", model,
        "--prompt", prompt
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.stdout.strip()

- 次に、テストデータに含まれる各サンプルに対して、ユーザが与えた命令・入力・正解出力の3つの情報を提示し、モデル応答（`model_response`）にスコアを付けるように依頼する

In [ ]:
from tqdm import tqdm

def generate_model_scores(json_data, json_key, model="llama3"):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = query_model(prompt, model)
        try:
            scores.append(int(score))
        except ValueError:
            print(f"Could not convert score: {score}")
            continue

    return scores

scores = generate_model_scores(test_data, "model_response")
print(f"Number of scores: {len(scores)} of {len(test_data)}")
print(f"Average score: {sum(scores)/len(scores):.2f}\n")

- 本モデルは平均スコア50を上回る結果となった。これは他モデルとの比較や学習設定を変更した際の参考値になる
- なお、OllamaはOSごとに完全な決定的挙動を示すわけではないため（本執筆時点）、若干異なる数値になる可能性がある

- 参考までに、オリジナルの
  - Llama 3 8B base model はスコア 58.51 を達成
  - Llama 3 8B instruct model はスコア 82.65 を達成

## 7.9 結論

### 7.9.1 次に進むには

- ここで本書は最終章となる
- これまでに、LLMアーキテクチャの実装・LLMの事前学習・ファインチューニングといったLLM開発サイクルの主要ステップをひととおり扱った

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/final-overview.webp?1" width=500px>

- 本章で示したインストラクションファインチューニングの後に、オプションとして好みや指向に合わせる "preference finetuning" が行われる場合がある
- Preferenceファインチューニングは特定のユーザ指向にモデルを合わせるために有用である。興味があれば [./04_preference-tuning-with-dpo](./04_preference-tuning-with-dpo) フォルダを参照してほしい

- 本リポジトリには、さらに多数のボーナス資料がある。詳細はリポジトリの README にある [Bonus Material](https://github.com/rasbt/LLMs-from-scratch?tab=readme-ov-file#bonus-material) を参照してほしい

### 7.9.2 変化の早い分野で最新情報を追うには

- このセクションにはコードはない

### 7.9.3 最後に

- 本書で、一からLLMを実装し、事前学習およびファインチューニングを行うという道のりを楽しんでもらえたら幸いである
- 個人的には、LLMをゼロから実装することがLLMの動作を理解するうえで最も効果的だと考えている。少しでも理解が深まっていれば嬉しい
- 教育を目的とした本書だが、実用面ではより強力なLLMを使いたい場合もあるだろう。
  - その場合、axolotl ([https://github.com/OpenAccess-AI-Collective/axolotl](https://github.com/OpenAccess-AI-Collective/axolotl)) や LitGPT ([https://github.com/Lightning-AI/litgpt](https://github.com/Lightning-AI/litgpt)) といったツールも検討するとよい。著者もこれらの開発に携わっている

## まとめと要点

- [./gpt_instruction_finetuning.py](./gpt_instruction_finetuning.py) スクリプトを参照。これは分類ファインチューニングのための1ファイル構成のサンプルである
- [./ollama_evaluate.py](./ollama_evaluate.py) は、セクション7.8をもとに、JSONファイル（"output" と "response" キーを含む）をOllamaとLlama 3で評価するスタンドアロンのスクリプトである
- [./load-finetuned-model.ipynb](./load-finetuned-model.ipynb) では、ファインチューニング後のモデルを新しいセッションで読み込む方法を示す
- [./exercise-solutions.ipynb](./exercise-solutions.ipynb) には、練習問題の解答例がある

## 今後の展望

- これで本書は完了である。追加資料を探しているならば、このリポジトリに複数のボーナスセクションを追加してあるので、ぜひ確認してほしい
- ボーナス資料の一覧は、メインREADMEの [Bonus Material](https://github.com/rasbt/LLMs-from-scratch?tab=readme-ov-file#bonus-material) セクションで確認できる
- 特に著者が気に入っているものをいくつか挙げる:
  1. [Direct Preference Optimization (DPO) for LLM Alignment (From Scratch)](../04_preference-tuning-with-dpo/dpo-from-scratch.ipynb): 本章のモデルをさらに人間の好みに合わせるように調整する人気のあるメカニズムを実装している
  2. [Llama 3.2 From Scratch (A Standalone Notebook)](../../ch05/07_gpt_to_llama/standalone-llama32.ipynb): Meta AIの人気モデルLlama 3.2をゼロから実装したもので、公式の事前学習済み重みを読み込む仕組みを含む。もし追加実験を行いたいなら、各章で使った `GPTModel` を `Llama3Model` に置き換える（1:1で差し替え可能）こともできる
  3. [Converting GPT to Llama](../../ch05/07_gpt_to_llama): GPT-2とLlama各種モデルの違いを段階的に解説したコードが含まれている
  4. [Embedding LayersとLinear Layersの違い](../../ch02/03_bonus_embedding-vs-matmul/embeddings-and-linear-layers.ipynb): PyTorchの `Embedding` 層（LLMの入力段階で使用）とone-hotエンコーディングからのlinear layerが数学的に同等であることを示す概念的な解説
- さらなる読書を楽しんでもらいたい